# Data Mining Process

En este notebook realizaremos todo el Data Mining Process. En el apartado 2. se pueden ver los pasos a realizar:


1. Entender el problema (Problem Understanding)
2. Data Mining Process
    * Obtener los datos (Data collect)
    * Procesando los datos (Data Processing)
    * Exploración estadística de datos (Statistical Data Exploration)
    * Modelado (Analysis Modeling)
3. Resultados (Results)



Antes de nada, importamos las librerias y funciones necesarias:

In [135]:
import tensorflow as tf
import pandas as pd
import skmob
from skmob.measures.individual import home_location
from skmob.io.file import load_geolife_trajectories,read,write


## 2.1. Obtener los datos

Antes de crear el modelo usando Tensorflow, es necesario cargar la información y preprocesarla para tenerla lista para entrenar el modelo de red neuronal que usaremos.

Para cargar los datos, usamos la función 'load_geolife_trajectories' que nos proporciona skmob. Esta función se encarga de cargar los datos de GeoLife Trajectories, solo hay que indicarle el path del directorio Data, y los usuarios que queremos cargar. Es opcional pasarle algunos parámetros de filtrado y compresión.Esta función, por cómo esta implementada, solo contempla la sintaxis de las rutas de archivos en Linux, por lo que en windows no funcionará.

Si queremos cargar/guardar estos datos, solo hay que hacer uso de las funciones read y write que proporciona skmob, que permiten escribir y cargar dataframes en formato JSON

In [136]:
# LOS DATOS YA ESTAN EN EL FICHERO
# LOS DATOS YA ESTAN EN EL FICHERO
# path = "/home/alonso/Documentos/tfg/Geolife_Trajectories/"
# traj_df = load_geolife_trajectories(path, user_ids=[120,121,122,123],
#                                    compress_kwargs={'spatial_radius_km': 0.0})

In [137]:
json_path= "../../Data/gps_data.json"
traj_df = read(json_path)


## 2.2. Procesando los datos

En esta segunda fase, es importante tener una "visión global" de los datos, con el fin de entenderlos mejor. Una buena idea es ver si hay datos nulos en la información que tenemos, ya que estos deberían ser tratados. Se puede ver que en esta ocasión, no los hay. Si los hubiese, una opción podría ser la de descartar estos datos. Para realizar esto último, se podría hacer uso del método dropna() para descartar las filas con algún dato nulo los datos. Otra opcion sería remplazar los nulos por algun otro valor con fillna()

In [138]:
df = pd.DataFrame(traj_df)
print('Total de valores nulos de cada columna: ')
print(df.isna().sum() )
print('\nVistazo de los datos:')
df.head()


Total de valores nulos de cada columna: 
lat         0
lng         0
datetime    0
uid         0
dtype: int64

Vistazo de los datos:


,lat,lng,datetime,uid
0,39.982375,116.320442,2009-09-19 07:11:37,120
1,39.982380,116.320448,2009-09-19 07:11:48,120
2,39.982380,116.320455,2009-09-19 07:11:50,120
3,39.982397,116.320455,2009-09-19 07:11:52,120
4,39.982425,116.320460,2009-09-19 07:11:54,120


Para obtener la localización de las casas de cada individuo en el dataset, scikit-mobility nos proporciona la función home_location(). Esta función observa la localización más frecuentada en la noche (Entre las 22:00 y las 07:00) por los usuarios y designa a este como su casa. Estas horas son modificables, aunque no lo haremos ya que lo normal, es estar en casa entre esas horas.

In [139]:
home_loc_df = home_location(df)
home_loc_df.head()

100%|██████████| 4/4 [00:00<00:00, 135.89it/s]


,uid,lat,lng
0,120,42.405338,117.249225
1,121,39.906183,116.379599
2,122,39.968092,116.399647
3,123,34.264762,108.939026


Como ya tenemos los dos dataframes, uno con los datos GPS y otro con las localizaciones de las casas de cada individuo, es necesario trabajar con ellos para obtener los datos que buscamos.

Añadimos al dataframe de los datos GPS (nuestro 'df') una columna extra que indicará si esta en casa y la inicializamos con 0. Al dataframe con la localizacion de las casas de los individuos (home_loc_df) le añadimos una columna extra de mismo nombre, con valor 1, ya que en esa posición se encuentra la casa.

Despues, hacemos lo que en base de datos se conoce como un JOIN LEFT usando como key los valores 'lat', 'lng' y 'uid'. Con conseguiremos tener la una columna at_home_y, con el valor de 1 cuando se encuentre una coincidencia y un valor nulo cuando no y otra at_home_x con todos sus valores 0. 

Por último ya solo quedaría crear y asignar a una nueva columna at_home el valor de la columna at_home_y y rellenar los valores nulos con los de la columna at_home_x (que tienen valor 0). Despues ya solo quedaría descartar las columnas que sobran y quedarmos con at_home

In [140]:
df['at_home'] = 0
home_loc_df['at_home'] = 1
df = pd.merge(df, home_loc_df, on=['lat','lng','uid'], how='left')
df['at_home'] = df['at_home_y'].fillna(df['at_home_x'])  
df = df.drop(['at_home_x','at_home_y'], axis=1)

df.head()

,lat,lng,datetime,uid,at_home
0,39.982375,116.320442,2009-09-19 07:11:37,120,0.0
1,39.982380,116.320448,2009-09-19 07:11:48,120,0.0
2,39.982380,116.320455,2009-09-19 07:11:50,120,0.0
3,39.982397,116.320455,2009-09-19 07:11:52,120,0.0
4,39.982425,116.320460,2009-09-19 07:11:54,120,0.0


## 2.3. Exploración estadística de datos

Observaremos las estadísticas de los datos y  comprobaremos si hay correlaciones entre ellos

In [141]:
print(df.dtypes)
df.describe()

lat                float64
lng                float64
datetime    datetime64[ns]
uid                  int64
at_home            float64
dtype: object


,lat,lng,uid,at_home
count,92830.000000,92830.000000,92830.000000,92830.000000
mean,38.559663,115.764088,121.804212,0.000118
std,4.675042,2.255154,0.751236,0.010885
min,22.534653,108.726840,120.000000,0.000000
25%,39.964671,116.231612,122.000000,0.000000
50%,39.976559,116.398177,122.000000,0.000000
75%,40.131145,116.474284,122.000000,0.000000
max,42.551380,117.957961,123.000000,1.000000


In [142]:
correlation_matrix =  df.corr()
matrix['at_home'].sort_values(ascending=False)

at_home    1.000000
lat        0.001342
lng       -0.003134
Name: at_home, dtype: float64

## 2.4. Modelado

Antes de crear el modelo de red neuronal con Tensorflow, es necesario normalizar las columnas, esto se hace para impedir que alguna feature influya más que otra en la predicción. Además, el hecho de tener los datos en valores entre 0 y 1, facilita bastante el entrenamiento. Para esto nos ayudaremos de sklearn y su método MinMaxScaler. mostando el dataframe nos damos cuenta que los datos se han normalizado correctamente.

In [143]:

## TO_DO: ENCONTRAR MANERA DE PREPROCESAR DATETIMES
## TO_DO: ENCONTRAR MANERA DE PREPROCESAR DATETIMES
df_preprocesed = df.drop('datetime', axis = 1)

In [144]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df_preprocesed) 
df_preprocesed.loc[:,:] = scaled_values
df_preprocesed.tail()



,lat,lng,uid,at_home
92825,0.584057,0.024237,1.0,0.0
92826,0.584059,0.024239,1.0,0.0
92827,0.584061,0.024239,1.0,0.0
92828,0.584074,0.024239,1.0,0.0
92829,0.584073,0.024239,1.0,0.0


Ahora debemos dividirlos en el set de entrenamiento y en el set de test, guardando la misma proporción de valores de la columna 'at_home' en ambos sets.

Para esto nos podemos ayudar de sklearn y su método StratifiedShuffleSplit. Para verificar que los datos se han separado en la misma proporción, basta con dividir tanto el número de 0 como de 1 entre el total de datos de cada set respectivamente.

In [145]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=10)

for train_index, test_index in split.split(df_preprocesed, df_preprocesed["at_home"]):
    strat_train_set = df_preprocesed.loc[train_index]
    strat_test_set = df_preprocesed.loc[test_index]

In [146]:
strat_test_set["at_home"].value_counts() / len(strat_test_set)

0.0    0.999892
1.0    0.000108
Name: at_home, dtype: float64

In [147]:
strat_train_set["at_home"].value_counts() / len(strat_train_set)

0.0    0.999879
1.0    0.000121
Name: at_home, dtype: float64


Ahora debemos separar la columna 'at_home' del resto de features en ambos conjuntos, ya que es la columna a predecir (nuestro target). Para ello usaremos pandas.

In [148]:
train_set = strat_train_set
test_set = strat_test_set
train_labels = train_set.pop('at_home')
test_labels = test_set.pop('at_home')

Ahora crearemos el modelo de Tensorflow 2.0 usando la API de alto nivel que incorpora llamada keras:

* La input layer no es una capa en sí, sinó un tensor, concretamente el tensor inicial que se manda a la primera capa oculta. 
* El tensor debe tener la misma shape(forma) que nuestros datos. En nuestro caso, este tensor es de 1D, con con tamaño igual al número de features de nuestro set de nuestro set de entrenamiento 'train_set'
* Se utilizarán dos hidden layer (hidden layer 1 y hidden layer 2) con 32 neuronas cada una, utilizando la función de activación 'relu'
* En cuanto a la ouput layer o capa de salida, estará compuesta de 1 neurona, ya que solo queremos predecir un dato. Como estamos haciendo una clasificación binaria, deberemos usar la función de activación 'sigmoid' que dará valores entre 0 y 1, lo que nos viene perfecto. 
* La función de perdida será la 'binary_crossentropy', que es la usada en problemas de clasificación binaria.






![red_neuronal_ejemplo](../../Images/neural_net.png)

In [149]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[len(train_set.keys())]),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation = 'sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

model = build_model()

Para hacer un resumen del modelo, podemos usar la función summary()

In [150]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 32)                128       
_________________________________________________________________
dense_21 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 33        
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


Ahora entrenamos el modelo durante 5 epochs y veremos como evoluciona

In [151]:
history = model.fit(
    train_set, 
    train_labels,
    epochs=5, 
    validation_split = 0.3)


Train on 51984 samples, validate on 22280 samples
Epoch 1/5
51984/51984 [==============================] - 7s 126us/sample - loss: 0.0153 - accuracy: 0.9998 - val_loss: 5.0296e-04 - val_accuracy: 1.0000
Epoch 2/5
51984/51984 [==============================] - 6s 109us/sample - loss: 0.0016 - accuracy: 0.9998 - val_loss: 4.8529e-04 - val_accuracy: 1.0000
Epoch 3/5
51984/51984 [==============================] - 6s 107us/sample - loss: 0.0016 - accuracy: 0.9998 - val_loss: 4.7882e-04 - val_accuracy: 1.0000
Epoch 4/5
51984/51984 [==============================] - 7s 130us/sample - loss: 0.0016 - accuracy: 0.9998 - val_loss: 4.6338e-04 - val_accuracy: 1.0000
Epoch 5/5
51984/51984 [==============================] - 6s 123us/sample - loss: 0.0015 - accuracy: 0.9998 - val_loss: 4.6936e-04 - val_accuracy: 1.0000


In [152]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


,loss,accuracy,val_loss,val_accuracy,epoch
0,0.015276,0.999846,0.000503,0.999955,0
1,0.001623,0.999846,0.000485,0.999955,1
2,0.001642,0.999846,0.000479,0.999955,2
3,0.001617,0.999846,0.000463,0.999955,3
4,0.001548,0.999846,0.000469,0.999955,4


Probamos el modelo en el test de entrenamiento

In [153]:
test_loss, test_acc = model.evaluate(test_set,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

18566/18566 - 1s - loss: 0.0013 - accuracy: 0.9999

Test accuracy: 0.9998923
